# **Simple RAG Application with LangChain and Gemini**

This project implements a Retrieval-Augmented Generation (RAG) application using LangChain and Google's Gemini LLM. It retrieves relevant information from a set of documents and generates answers to user queries.

## Features
- Document chunking and embedding using HuggingFace's `all-MiniLM-L6-v2`.
- FAISS vector store for efficient similarity search.
- Integration with Gemini LLM (`gemini-2.5-flash`) for answer generation.
- Simple query interface for retrieving and answering based on document context.

## Prerequisites
- Python 3.8+
- Google API key for Gemini LLM
- Required packages: `langchain`, `langchain-community`, `langchain-google-genai`, `faiss-cpu`, `python-dotenv`, `sentence-transformers`

## Installation
1. Install dependencies:
   ```bash
   pip install langchain langchain-community langchain-google-genai faiss-cpu python-dotenv sentence-transformers
   ```
2. Create a `.env` file in the project root and add your Google API key:
   ```
   GOOGLE_API_KEY=your_google_api_key_here
   ```

## Customization
- Replace `sample_docs` with your own data (e.g., from files or databases).
- Use alternative embedding models or vector stores for better performance.
- Adjust the `chunk_size` or `search_kwargs` in `rag_application.py` for different retrieval behavior.


# Install dependencies

In [1]:
!pip install langchain langchain-community langchain-google-genai langchain-pinecone pinecone-client python-dotenv sentence-transformers

In [28]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 26.6 MB/s eta 0:00:00


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os

# Load environment variables

In [3]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Sample documents

In [7]:
sample_docs = [
    "The capital of France is Paris. It is known for its cafes, museums, and the Eiffel Tower.",
    "Python is a versatile programming language used for web development, data science, and automation.",
    "The theory of relativity was developed by Albert Einstein in the early 20th century."
]

# Convert Document and create vector database using FAISS

In [8]:
# Convert sample documents to LangChain Document objects
documents = [Document(page_content=doc) for doc in sample_docs]

In [15]:
print(f"Number of documents: {len(documents)}")
print(f"First document: {documents[0]}")

Number of documents: 3
First document: page_content='The capital of France is Paris. It is known for its cafes, museums, and the Eiffel Tower.'


In [23]:
# Split documents into chunks
text_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
doc_chunks = text_spliter.split_documents(documents)

In [24]:
print(f"Number of chunks: {len(doc_chunks)}")
print(f"First chunk: {doc_chunks[0].page_content}")

Number of chunks: 3
First chunk: The capital of France is Paris. It is known for its cafes, museums, and the Eiffel Tower.


In [25]:
# Initialize embeddings (using HuggingFace model)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-2903832910.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [29]:
# Create FAISS vector store
vector_store = FAISS.from_documents(doc_chunks, embeddings)

In [35]:
vector_store.similarity_search("france")

[Document(id='a5160f1f-ed0a-41a8-ae6d-bbbe122641df', metadata={}, page_content='The capital of France is Paris. It is known for its cafes, museums, and the Eiffel Tower.'),
 Document(id='ef9b77bb-1fde-449f-8331-37da03d28263', metadata={}, page_content='The theory of relativity was developed by Albert Einstein in the early 20th century.'),
 Document(id='1134bf42-c99a-4cca-99d9-71845b442ce9', metadata={}, page_content='Python is a versatile programming language used for web development, data science, and automation.')]

# Retrieval + Gemini LLM

In [36]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash', api_key=GOOGLE_API_KEY)

In [38]:
# Setup retrievalQA chain

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
)

# Tests

In [40]:
query = "What is the capital of France?"

In [41]:
result = qa_chain.invoke({"query": query})

In [42]:
result

{'query': 'What is the capital of France?',
 'result': 'The capital of France is Paris.',
 'source_documents': [Document(id='a5160f1f-ed0a-41a8-ae6d-bbbe122641df', metadata={}, page_content='The capital of France is Paris. It is known for its cafes, museums, and the Eiffel Tower.'),
  Document(id='1134bf42-c99a-4cca-99d9-71845b442ce9', metadata={}, page_content='Python is a versatile programming language used for web development, data science, and automation.')]}

In [44]:
query1 = "What are particularities of France capital Paris?"
result1 = qa_chain.invoke({"query": query1})
result1

{'query': 'What are particularities of France capital Paris?',
 'result': 'Paris is known for its cafes, museums, and the Eiffel Tower.',
 'source_documents': [Document(id='a5160f1f-ed0a-41a8-ae6d-bbbe122641df', metadata={}, page_content='The capital of France is Paris. It is known for its cafes, museums, and the Eiffel Tower.'),
  Document(id='1134bf42-c99a-4cca-99d9-71845b442ce9', metadata={}, page_content='Python is a versatile programming language used for web development, data science, and automation.')]}

In [45]:
query2 = "What are use cases of python?"
result2 = qa_chain.invoke({"query": query2})
result2

{'query': 'What are use cases of python?',
 'result': 'Python is used for web development, data science, and automation.',
 'source_documents': [Document(id='1134bf42-c99a-4cca-99d9-71845b442ce9', metadata={}, page_content='Python is a versatile programming language used for web development, data science, and automation.'),
  Document(id='ef9b77bb-1fde-449f-8331-37da03d28263', metadata={}, page_content='The theory of relativity was developed by Albert Einstein in the early 20th century.')]}